## Export Neo4J database to Json
> Make sure the there is data in database

In [1]:
import os
import json

from neo4j import GraphDatabase
from langchain_community.graphs import Neo4jGraph

from dotenv import load_dotenv

In [2]:
# load dotenv
load_dotenv()
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
NEO4J_DATABASE = os.getenv("NEO4J_DATABASE")

### Connect to database

In [3]:
graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database=NEO4J_DATABASE
)

In [4]:
# update the schema of graph(if changes are made in graph)
graph.refresh_schema()
print(graph.schema)

Node properties:
Chunk {chunkSeqId: INTEGER, chunkId: STRING, cusip6: STRING, id: INTEGER, text: STRING, names: LIST, cik: STRING, source: STRING, formId: STRING, f10kItem: STRING, textEmbedding: LIST}
Form {cusip6: STRING, source: STRING, id: INTEGER, formId: STRING, names: LIST}
Company {cusip6: STRING, location: LIST, companyName: STRING, names: LIST, id: INTEGER, cusip: STRING, companyAddress: STRING}
Manager {managerName: STRING, managerCik: STRING, managerAddress: STRING, id: INTEGER, location: LIST}
Address {country: STRING, id: INTEGER, location: LIST, city: STRING, state: STRING}
Relationship properties:
SECTION {f10kItem: STRING}
OWNS_STOCK_IN {value: FLOAT, shares: INTEGER, reportCalendarOrQuarter: STRING}
The relationships:
(:Chunk)-[:NEXT]->(:Chunk)
(:Chunk)-[:PART_OF]->(:Form)
(:Form)-[:SECTION]->(:Chunk)
(:Company)-[:LOCATED_AT]->(:Address)
(:Company)-[:FILED]->(:Form)
(:Manager)-[:LOCATED_AT]->(:Address)
(:Manager)-[:OWNS_STOCK_IN]->(:Company)


In [5]:
# get manager and their address
graph.query("""
MATCH (mgr:Manager)-[:LOCATED_AT]->(addr:Address)
RETURN mgr, addr
LIMIT 1
""")

[{'mgr': {'managerCik': '1424381',
   'managerAddress': '650 Madison Avenue, 25th Floor, New York, NY, 10022',
   'location': [-73.9713457, 40.7639879],
   'id': 1145,
   'managerName': 'LAKEWOOD CAPITAL MANAGEMENT, LP'},
  'addr': {'country': 'United States',
   'city': 'New York',
   'location': [-73.9821429, 40.7584882],
   'id': 3302,
   'state': 'New York'}}]

In [6]:
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

query = """
MATCH (n)
OPTIONAL MATCH (n)-[r]->(m)
RETURN n, r, m
"""

In [7]:
# Initialize the Neo4j driver
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

query = """
MATCH (n)
OPTIONAL MATCH (n)-[r]->(m)
RETURN n, r, m
"""

# Run the query and process results
with driver.session() as session:
    result = session.run(query)
    export_data = []
    for record in result:
        node_data = record.get('n', None)
        relationship_data = record.get('r', None)
        connected_node_data = record.get('m', None)

        # Process data as needed
        export_data.append({
            'node': dict(node_data) if node_data else None,
            'relationship': dict(relationship_data) if relationship_data else None,
            'connected_node': dict(connected_node_data) if connected_node_data else None
        })

WRITE_FILE_PATH = 'exported_data.json'
# Save data to a JSON file
with open(WRITE_FILE_PATH, 'w') as f:
    json.dump(export_data, f, indent=4)

# Close the driver connection
driver.close()